In [1]:
!pip install matplotlib numpy pandas plotly scipy tqdm Brian2

     ---------------------------------------- 0.0/57.6 kB ? eta -:--:--
     ------- -------------------------------- 10.2/57.6 kB ? eta -:--:--
     ---------------------------------------- 57.6/57.6 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/16.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/16.4 MB 5.0 MB/s eta 0:00:04
   ---------------------------------------- 0.2/16.4 MB 5.0 MB/s eta 0:00:04
    --------------------------------------- 0.2/16.4 MB 1.5 MB/s eta 0:00:11
   - -------------------------------------- 0.7/16.4 MB 3.6 MB/s eta 0:00:05
   -- ------------------------------------- 1.0/16.4 MB 4.3 MB/s eta 0:00:04
   --- ------------------------------------ 1.5/16.4 MB 5.1 MB/s eta 0:00:03
   ---- ----------------------------------- 2.0/16.4 MB 5.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.5/16.4 MB 6.3 MB/s eta 0:00:03
   ------- -------------------------------- 3.0/16.4 MB 7.1 MB/s eta 0:00:02
   -------- 

In [1]:
import os
from brian2 import *
import random
from tqdm.notebook import tqdm
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from plots import *
from equations import *
from global_settings import *
from masks import *
from helper import *
from run_loop import *

import time

Function to create a random 3D topology, with a total number of neurons N and spatial bounds.
- N: integer representing the total number of neurons to generate.
- bounds: A tuple or list containing three elements that define the maximum bounds for the x, y, and z dimensions of the space where the neurons will be placed.

the numbers are uniformly distributed between 0 and the 3D area (determined by the bound)

This results in three lists: x, y, and z, each containing N elements. Each element represents the coordinate along one axis for a neuron.

the topology (= Netzstruktur) array contains the randomly generated 3D positions of all N neurons within the specified bounds


In [2]:
def create_neuron_topology (N, bounds):
    
    x_bound, y_bound, z_bound = bounds # unpacks the bounds parameter into three variables
    
    x = [random.uniform(0, x_bound) for _ in range(N)]
    y = [random.uniform(0, y_bound) for _ in range(N)]
    z = [random.uniform(0, z_bound) for _ in range(N)]
    topology = np.array([x, y, z])
    
    return topology

The `create_group_py` function is designed to create a group of excitatory neurons using Brian2, a Python library for simulating spiking neural networks.

1. **Function Parameters**:
   - `topology`: A NumPy array containing the 3D coordinates (x, y, z) of each neuron.
   - `noise`: A tuple containing two elements, `mu_noise` and `sigma_noise`, which represent the mean and standard deviation of the noise applied to the neurons.
   - `masks`: A tuple containing two elements, `stimulus_mask` and `treatment_mask`, which are likely boolean arrays indicating whether each neuron should receive a stimulus or undergo a specific treatment.
   - `group_name`: A string specifying the name of the neuron group. Default is `'exc_group'`.
   - `integ_method`: A string specifying the integration method to use for updating neuron states. Default is `'exponential_euler'`.

2. **Extracting Parameters**: The function begins by extracting the x, y, z coordinates from the `topology` array and the mean and standard deviation of the noise from the `noise` tuple. It also separates the `stimulus_mask` and `treatment_mask` from the `masks` tuple.

3. **Creating Neuron Group**: Using Brian2's `NeuronGroup` class, it creates a group of excitatory neurons (`G_exc`) based on the length of the x-coordinate list (which equals the number of neurons). The neurons are defined by equations (`py_eqs`), a threshold condition (`'v>V_th'`), reset rules (`reset_eqs`), a refractory period (`3*ms`), and a specified integration method.

4. **Initializing Neuron Properties**:
   - Initial membrane potential (`v`) is set to `-60*mvolt` minus a random value between 0 and `40*mvolt` for each neuron, introducing variability.
   - Glutamate concentration (`glu`) is initialized to 1 for all neurons.
   - Spatial coordinates (`x`, `y`, `z`) are assigned from the `topology` array, converting them to micrometers (`um`) since Brian2 requires physical units.
   - A custom attribute `taille` (possibly representing size or another property) is set to a predefined value `taille_exc_normale`.
   - Noise parameters (`mu_noise`, `sigma_noise`) are assigned to each neuron.
   - Treatment and stimulus masks are assigned to each neuron, allowing selective application of treatments or stimuli.

5. **Return Value**: The function returns the created `NeuronGroup` object (`G_exc`), which now represents a group of excitatory neurons with specified properties and initial conditions ready for simulation.

In essence, this function configures a group of excitatory neurons with specific characteristics, including their spatial locations, initial states, and parameters for noise and masking, preparing them for inclusion in a larger neural network simulation using Brian2.

In [3]:
def create_group_py(topology, noise, masks, group_name='exc_group', integ_method='exponential_euler'):

    x, y, z = topology
    mu_noise, sigma_noise = noise
    stimulus_mask, treatment_mask = masks
        
    G_exc=NeuronGroup(len(x),py_eqs,threshold='v>V_th',reset=reset_eqs,refractory=3*ms,name=group_name, method=integ_method)
    G_exc.v = '-60*mvolt-rand()*40*mvolt'
    G_exc.glu = 1
    G_exc.x = x * um
    G_exc.y = y * um
    G_exc.z = z * um

    G_exc.taille=taille_exc_normale

    G_exc.mu_noise = mu_noise
    G_exc.sigma_noise = sigma_noise

    G_exc.treatment_mask = treatment_mask
    G_exc.stimulus_mask = stimulus_mask

    return G_exc

The `create_group_inh` function is similar to the previously described `create_group_py` but is specifically tailored for creating a group of inhibitory neurons using the Brian2 library. Here's a breakdown of its functionality:

1. **Function Parameters**:
   - `topology`: A NumPy array containing the 3D coordinates (x, y, z) of each neuron.
   - `noise`: A tuple containing two elements, `mu_noise` and `sigma_noise`, which represent the mean and standard deviation of the noise applied to the neurons.
   - `treatment_mask`: A boolean array indicating whether each neuron should undergo a specific treatment.
   - `group_name`: A string specifying the name of the neuron group. Default is `'inh_group'`.
   - `integ_method`: A string specifying the integration method to use for updating neuron states. Default is `'exponential_euler'`.

2. **Extracting Parameters**: The function extracts the x, y, z coordinates from the `topology` array and the mean and standard deviation of the noise from the `noise` tuple.

3. **Creating Neuron Group**: It creates a group of inhibitory neurons (`G_inh`) using Brian2's `NeuronGroup` class. The number of neurons is determined by the length of the x-coordinate list. The neurons are defined by equations (`inh_eqs`), a threshold condition (`'v>V_th'`), a refractory period (`3*ms`), and a specified integration method.

4. **Initializing Neuron Properties**:
   - Initial membrane potential (`v`) is set to `-60*mvolt` minus a random value between 0 and `10*mvolt` for each neuron, adding variability.
   - A custom attribute `taille` (possibly representing size or another property) is set to a predefined value `taille_inh_normale`.
   - Spatial coordinates (`x`, `y`, `z`) are assigned from the `topology` array, converting them to micrometers (`um`).
   - Noise parameters (`mu_noise`, `sigma_noise`) are assigned to each neuron.
   - The treatment mask is assigned to each neuron, allowing selective application of treatments.

5. **Return Value**: The function returns the created `NeuronGroup` object (`G_inh`), which represents a group of inhibitory neurons with specified properties and initial conditions, ready for simulation.

In summary, this function sets up a group of inhibitory neurons with specific characteristics, including their spatial locations, initial states, and parameters for noise and treatment masking, preparing them for inclusion in a neural network simulation using Brian2.


In [4]:
def create_group_inh(topology, noise, treatment_mask, group_name='inh_group', integ_method='exponential_euler'):

    x, y,z = topology
    mu_noise, sigma_noise = noise
        
    G_inh=NeuronGroup(len(x),inh_eqs,threshold='v>V_th', name=group_name, refractory=3*ms,method=integ_method)
    G_inh.v = -60*mvolt-rand()*10*mvolt

    G_inh.taille=taille_inh_normale
    G_inh.x = x * um
    G_inh.y = y * um
    G_inh.z = z * um

    
    G_inh.mu_noise = mu_noise
    G_inh.sigma_noise = sigma_noise

    G_inh.treatment_mask = treatment_mask

    return G_inh

The `create_group_lfp` function is designed to create a Local Field Potential (LFP) recording site within a simulated environment using the Brian2 library. Here's a detailed explanation of its purpose and operation:

1. **Function Parameter**:
   - `bounds`: A tuple or list containing three elements that define the maximum bounds for the x, y, and z dimensions of the space where the LFP electrode will be placed.

2. **Bounds Extraction**: It unpacks the `bounds` parameter into three variables: `x_bound`, `y_bound`, and `z_bound`. These represent the maximum values for the x, y, and z coordinates within which the LFP electrode position will be determined.

3. **Setting Up the LFP Electrode**:
   - The function initializes a single LFP electrode (`Ne = 1`) with a resistivity (`sigma`) of the extracellular field, typically ranging between 0.3 to 0.4 Siemens per meter (S/m). This resistivity value is crucial for modeling the electrical conductivity of the brain tissue surrounding the electrode.
   
4. **Creating the NeuronGroup for LFP**:
   - It creates a `NeuronGroup` with just one neuron (`Ne = 1`) to represent the LFP electrode. This group is defined with a simple model that includes a voltage variable (`v`) and spatial coordinates (`x`, `y`, `z`) in meters. Although it's called a "neuron" group, in this context, it serves as a placeholder for the LFP electrode.
   
5. **Positioning the LFP Electrode**:
   - The LFP electrode is positioned at the center of the defined bounds by setting its `x`, `y`, and `z` coordinates to half the values of `x_bound`, `y_bound`, and `z_bound`, respectively. These coordinates are converted to micrometers (`um`) because Brian2 requires physical quantities to have units.

6. **Return Value**: The function returns the created `NeuronGroup` object (`lfp`), which represents the LFP electrode placed at the specified location within the simulation space.

In essence, this function prepares a single LFP recording site at the center of a given 3D space, ready to be integrated into a larger neural network simulation. The LFP electrode can then be used to record the collective electrical activity of nearby neurons, providing insights into the network dynamics.

In [5]:
def create_group_lfp (bounds):

    x_bound, y_bound, z_bound = bounds
    
    # Set up a singular LFP electrode
    Ne = 1
    sigma = 0.3*siemens/meter # Resistivity of extracellular field (0.3-0.4 S/m)
    lfp = NeuronGroup(Ne, model='''v : volt
                                   x : meter
                                   y : meter
                                   z : meter''')
    lfp.x = x_bound/2*um
    lfp.y = y_bound/2*um
    lfp.z = z_bound/2*um

    return lfp

The `read_input_signal` function reads an external file containing signal data and converts it into a `TimedArray` object suitable for use in Brian2 simulations. Here's a step-by-step explanation:

1. **Function Parameter**:
   - `file_name`: A string specifying the name of the file containing the input signal data. The file is expected to be located in a subdirectory named `./stimuli/`.

2. **Loading Data**:
   - The function uses `np.loadtxt` to load numerical data from the specified file. This data is assumed to represent the input signal over time, stored in a text file. The loaded data is stored in the variable `in_1`.

3. **Creating a TimedArray Object**:
   - The loaded data (`in_1`) is multiplied by `namp` to convert the signal amplitude to nanoamperes (nA), assuming `namp` is a conversion factor defined elsewhere in the code.
   - A `TimedArray` object named `input_signal` is created using the modified data. `TimedArray` is a Brian2 class that allows for the specification of time-varying inputs in simulations. The `dt` parameter specifies the time step for the array, which is set to `defaultclock.dt`, the default time step used by Brian2 for simulations.

4. **Return Value**:
   - The function returns the `input_signal` `TimedArray` object, which can now be used in Brian2 simulations to provide time-varying input currents to neurons or other components.

In summary, this function facilitates the incorporation of externally defined signals into Brian2 simulations by reading the signal data from a file, scaling it appropriately, and packaging it into a `TimedArray` object for easy integration into the simulation framework.


In [6]:
def read_input_signal (file_name):
    in_1 = np.loadtxt('./stimuli/'+file_name)

    input_signal = TimedArray(in_1*namp,dt=defaultclock.dt)

    return input_signal


The `prepare_network` function sets up a neural network simulation using the Brian2 library, incorporating both excitatory and inhibitory neuron populations, synaptic connections, and monitoring tools. Here's a detailed breakdown of its operations:

1. **Parameter Expansion**:
   - Extracts connection probabilities (`p_e2e`, `p_e2i`, `p_i2e`, `p_i2i`) and other variables from the `current_variables` dictionary.
   - Separates topologies for excitatory and inhibitory neurons (`topology_exc`, `topology_inh`) and treatment masks (`treatment_mask_exc`, `treatment_mask_inh`) from the provided arguments.

2. **Creating Neuron Groups**:
   - Calls `create_group_py` to create a group of excitatory neurons (`G_exc`) with specified topologies, noise parameters, and masks.
   - Calls `create_group_inh` to create a group of inhibitory neurons (`G_inh`) with specified topologies and noise parameters.
   - Calls `create_group_lfp` to create a Local Field Potential (LFP) recording site (`G_lfp`) within the simulation space.

3. **Setting Up Monitors**:
   - Initializes population rate monitors (`popmon_exc`, `popmon_inh`) to track the firing rates of excitatory and inhibitory neurons.
   - Sets up state monitors (`statemon_exc`, `statemon_inh`) to record membrane potentials and current values for selected neurons in both populations.
   - Configures a state monitor (`Mlfp`) to record the voltage at the LFP electrode.
   - Collects all monitors into a list for later access.

4. **Defining Synapses and Connections**:
   - Creates synaptic connections between excitatory neurons (`S_e2e`), from excitatory to inhibitory neurons (`S_e2i`), and from inhibitory to excitatory neurons (`S_i2e`). The connection probability and weight formulas incorporate distance-dependent decay.
   - Establishes a special type of synapse (`S_lfp`) to calculate the contribution of excitatory neurons to the LFP signal, considering the resistivity of the medium and the distance between neurons and the LFP electrode.

5. **Network Assembly**:
   - Combines neuron groups, synapses, and monitors into a Brian2 `Network` object (`net`), which encapsulates the entire simulation setup.

6. **Return Values**:
   - Returns the assembled `Network` object, along with lists of synapses and monitors, ready for running the simulation.

In summary, this function orchestrates the creation of a complex neural network model involving excitatory and inhibitory neurons, their synaptic interactions, and mechanisms for recording network activity and LFP signals.

In [7]:
def prepare_network (topologies, stimulus_mask_exc, treatment_masks, current_variables):

    # Expand parameter lists
    p_e2e, p_e2i, p_i2e, p_i2i = current_variables['p']
    topology_exc, topology_inh = topologies
    treatment_mask_exc, treatment_mask_inh = treatment_masks
    
    # Create py and inh groups
    G_exc = create_group_py(topology_exc, current_variables['noise_exc'], [stimulus_mask_exc, treatment_mask_exc])
    G_inh = create_group_inh(topology_inh, current_variables['noise_inh'], treatment_mask_inh)
    G_lfp = create_group_lfp(current_variables['bounds'])
    
    # Set up monitors
    popmon_exc = PopulationRateMonitor(G_exc)
    popmon_inh = PopulationRateMonitor(G_inh)
    Mlfp = StateMonitor(G_lfp, 'v', record=True)
    statemon_exc = StateMonitor(G_exc, ('v', 'I_stim', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    statemon_inh = StateMonitor(G_inh, ('v', 'I_noise'), record=[1,2,3,4,5,6], dt=0.001*second)
    monitors = [popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp]
    neuron_groups = [G_exc, G_inh, G_lfp]
    
    # Define synapses and connect neuron groups
    S_e2e=Synapses(G_exc,   G_exc, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre")
    S_e2i = Synapses(G_exc, G_inh, on_pre="he_post+="+str(gain)+"*"+str(g_max_e/siemens)+"*siemens*glu_pre", name='synapses_e2i')
    S_i2e = Synapses(G_inh, G_exc, on_pre="hi_post+="+str(gain)+"*"+str(g_max_i/siemens)+"*siemens", name='synapses_i2e')

    S_e2e.connect(p=f'{p_e2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_e2i.connect(p=f'{p_e2i}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({489.9}*umeter)**2)))')
    S_i2e.connect(p=f'{p_i2e}*exp(-(((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2) / (({215}*umeter)**2)))')

    S_lfp = Synapses(G_exc, G_lfp, model='''w : ohm*meter**2 (constant) # Weight in the LFP calculation
                                       v_post = w*((0.0*amp/meter**2)-Im_pre) : volt (summed)''')
    S_lfp.summed_updaters['v_post'].when = 'after_groups'  # otherwise Ic has not yet been updated for the current time step.
    S_lfp.connect()
    S_lfp.w = '(29e3 * umetre ** 2)/(4*pi*sigma)/((x_pre-x_post)**2+(y_pre-y_post)**2+(z_pre-z_post)**2)**.5'


    synapses = [S_e2e, S_e2i, S_i2e, S_lfp]

    net = Network(neuron_groups, synapses, monitors)

    return net, synapses, monitors

The `run_granular_simulation` function executes a granular layer neural network simulation using the Brian2 library, with dynamic adjustments based on the network's activity. Here's a step-by-step explanation:

1. **Print Start Message**: Displays a message indicating the start of the simulation process.

2. **Simulation Setup**:
   - Retrieves the total duration of the simulation and the name of the file containing the input signal from the `variables` dictionary.
   - Reads the input signal using the `read_input_signal` function, which loads the signal data from a file and converts it into a `TimedArray` object for use in the simulation.
   - Extracts treatment settings (`time_fragment`, `firing_rate_threshold`) and monitors from the provided arguments.

3. **Baseline Excitability Settings**:
   - Retrieves baseline excitability levels for excitatory (`Eke_baseline`) and inhibitory (`Eki_baseline`) neurons from the `variables` dictionary.

4. **Print Treatment Parameters**: Outputs the treatment parameters being used in the simulation, including the time fragment for evaluating network activity and the firing rate threshold for triggering changes.

5. **Simulation Loop**:
   - Calculates the number of batches (`num_batches`) by dividing the total simulation duration by the time fragment.
   - Iterates over each batch using a progress bar (`tqdm`) for visual feedback during execution.
   - Runs the simulation for each time fragment using `net.run(time_fragment)`.

6. **Dynamic Adjustment Based on Firing Rate**:
   - After each time fragment, checks if the average firing rate of the excitatory population (`popmon_exc.rate`) exceeds the specified threshold (`firing_rate_threshold`) over the last time fragment.
   - If the threshold is exceeded, adjusts the excitability levels of excitatory (`Eke`) and inhibitory (`Eki`) neurons according to the treatment settings specified in the `variables` dictionary.

7. **Loop Continuation**: Repeats steps 5 and 6 for each batch until the total simulation duration is reached.

In summary, this function runs a neural network simulation in segments, dynamically adjusting neuronal excitability based on the observed firing rate of the excitatory population. This approach allows for real-time modulation of network behavior in response to its activity patterns, mimicking certain aspects of plasticity or adaptive processes in biological neural systems.

In [8]:
def run_granular_simulation (net, variables, treatment_settings, monitors):
    
    print('#######################')
    print('# Starting Simulation #')
    print('#######################')
    print()
    
    total_duration = variables['duration']
    input_signal = read_input_signal(variables['input_signal_file'])
    
    time_fragment, firing_rate_threshold = treatment_settings
    popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
    
    Eke = variables['Eke_baseline']
    Eki = variables['Eki_baseline']
    Eke_baseline = variables['Eke_baseline']
    Eki_baseline = variables['Eki_baseline']
    
    # Print Parameters Used
    print('Treatment Parameters:', 'time sensitivity', time_fragment, 'FR Threshold:', firing_rate_threshold)
    print()
    
    time_fragment_ms = int(time_fragment/ms)
    num_batches = int(total_duration / time_fragment)
    
    for i in tqdm(range(num_batches), desc="Running Simulation"):

        # Run the Simulation 
        net.run(time_fragment)

        if np.mean(popmon_exc.rate[-time_fragment_ms:]) > firing_rate_threshold:
            Eke = variables['Eke_treatment']
            Eki = variables['Eki_treatment']

The `run_model_loop` function orchestrates the execution of multiple neural network simulations, each with potentially different parameters, and manages the output of these simulations. Here's a detailed breakdown of its operations:

1. **Validation Check**: Ensures that all variable lists within the `variables` dictionary have equal lengths using the `check_dict_lengths` function. If not, raises a `ValueError`.

2. **Electrode Position Initialization**: Populates the positions of electrodes across all runs using the `populate_electrode_positions` function and stores these positions in the `variables` dictionary under `"coord_of_electrode"`.

3. **Simulation Loop**:
   - Iterates over each set of parameters defined in the `variables` dictionary, indexed by the `run_id`.
   - Resets the Brian2 scope and sets the default clock time step to `0.001*second` for precise timing control.

4. **Current Variables Extraction**: Creates a subset of the `variables` dictionary for the current iteration, containing only the parameters relevant to the current run.

5. **Setup for Each Run**:
   - Creates a unique directory for storing the results of the current run.
   - Writes the run settings to a file within the newly created directory.
   - Generates topologies for excitatory and inhibitory neurons using the `create_neuron_topology` function.
   - Creates spherical masks for stimulus and treatment applications based on the specified geometries.
   - Prepares treatment masks for both excitatory and inhibitory neurons, initially setting all neurons to be treated.

6. **Network Preparation and Execution**:
   - Calls `prepare_network` to instantiate the neural network with the current parameters, including neuron groups, synapses, and monitors.
   - Writes network statistics and saves plots of neuron masks and treatment areas to the results directory.

7. **Simulation Execution**:
   - Executes the simulation using `run_granular_simulation`, passing the prepared network, current variables, treatment settings, and monitors.
   - Saves the firing rate data for both excitatory and inhibitory populations to text files.

8. **Post-Simulation Analysis and Visualization**:
   - Plots and saves figures showing the firing rates of excitatory and inhibitory neurons, noise levels, and the local field potential (LFP) recorded by the LFP electrode.

9. **Exception Handling**: Catches any exceptions that occur during the simulation or analysis phases, prints a message indicating a broken run, and continues with the next iteration after a brief pause.

In summary, this function automates the process of running a series of neural network simulations with varying parameters, managing the organization of results, and generating comprehensive reports including network statistics, firing rates, noise levels, and LFP recordings.

In [9]:
def run_model_loop (variables):

    if not check_dict_lenghts(variables):
        raise ValueError('Lenghts of each variable list has be to equal!')

    variables["coord_of_electrode"] = populate_electrode_positions(variables)

    for i in range(len(variables['run_id'])):
        
        start_scope()
        defaultclock.dt = 0.001*second   

        current_variables = {key: variables[key][i] for key in variables}

        # Create folder
        run_id = current_variables["run_id"]
        os.mkdir(f'./results/{run_id}')
        write_run_settings(current_variables, run_id)
        
        print(current_variables['noise_exc'])

        # Create dynamic objects based on variables provided
        topology_exc = create_neuron_topology(current_variables['N'][0], current_variables['bounds'])
        topology_inh = create_neuron_topology(current_variables['N'][1], current_variables['bounds'])
        topologies = [topology_exc, topology_inh]
        
        stimulus_geometry_settings = [current_variables['coord_of_stimulus'], current_variables['radius_of_stimulus']]
        stimulus_mask_exc = create_spherical_mask(topology_exc, stimulus_geometry_settings)
        
        treatment_geometry_settings = [current_variables['coord_of_electrode'], current_variables['radius_of_electrode']]
        #treatment_mask_exc = create_spherical_mask(topology_exc, treatment_geometry_settings)
        #treatment_mask_inh = create_spherical_mask(topology_inh, treatment_geometry_settings)
        treatment_mask_exc = np.ones(current_variables['N'][0])
        treatment_mask_inh = np.ones(current_variables['N'][1])
        treatment_masks = [treatment_mask_exc, treatment_mask_inh]
        
        treatment_settings = [current_variables['device_sensitivity'], current_variables['firing_rate_threshold']]
        
        # Instantiate the Network
        net, synapses, monitors = prepare_network(topologies, stimulus_mask_exc, treatment_masks, current_variables)
        popmon_exc, popmon_inh, statemon_exc, statemon_inh, Mlfp = monitors
        
        # Write network statistics
        write_network_statistics(synapses, current_variables['N'], run_id)

        # Save network plots
        plot_neuron_masks (topology_exc, [stimulus_mask_exc, treatment_mask_exc], run_id)
        plot_neuron_mask (topology_exc, stimulus_mask_exc, 'red', 'stimulus', run_id)
        plot_neuron_mask (topology_exc, treatment_mask_exc, 'blue', 'treatment', run_id)

        try:
            # Run the simulation
            run_granular_simulation (net, current_variables, treatment_settings, monitors)

            # Save Firing Rate Data
            np.savetxt(f'./results/{run_id}/fr_exc.txt', popmon_exc.rate)
            np.savetxt(f'./results/{run_id}/fr_inh.txt', popmon_inh.rate)

            #Plot the Firing Rate and Noise

            plt.plot(popmon_inh.t, popmon_inh.rate, label='Inhibitory')
            plt.plot(popmon_exc.t, popmon_exc.rate, label='Excitatory')

            plt.legend()
            plt.savefig(f'./results/{run_id}/firing-rates.png', bbox_inches='tight')
            plt.close()

            plt.plot(statemon_inh.t, statemon_inh.I_noise[4]/nA, label='inh')
            plt.savefig(f'./results/{run_id}/noise_inh.png', bbox_inches='tight')
            plt.close()

            plt.plot(statemon_exc.t, statemon_exc.I_noise[4]/nA, label='exc')
            plt.savefig(f'./results/{run_id}/noise_exc.png', bbox_inches='tight')
            plt.close()

            np.savetxt(f'./results/{run_id}/LFP.txt', Mlfp.v[0]/mV)
            plot(Mlfp.t/ms, Mlfp.v[0]/mV)
            plt.savefig(f'./results/{run_id}/LFP.png', bbox_inches='tight')
            plt.close()

            
        except:
            print('Broken run.')
        
        time.sleep(1)
        

This code snippet defines a dictionary named `variables` that contains configuration settings for running a series of neural network simulations. Each key-value pair in the dictionary specifies a particular aspect of the simulation environment, such as network parameters, stimulus settings, and treatment conditions. The dictionary is designed to be used in a loop, where each iteration corresponds to a separate simulation run with slightly modified parameters. Here's a breakdown of the key components:

### General Settings
- **`run_id`**: A list of identifiers for each simulation run, presumably used to organize results.
- **`duration`**: The duration of each simulation run, repeated `copy_times` times.
- **`bounds`**: The bounds of the simulation space, repeated `copy_times` times, specifying the x, y, and z dimensions.

### Network Settings
- **`N`**: The number of excitatory and inhibitory neurons in the network, repeated `copy_times` times.
- **`Eke_baseline`**, **`Eki_baseline`**: Baseline excitability levels for excitatory and inhibitory neurons, repeated `copy_times` times. Setting `Eke_baseline` to `-90 mV` activates an "epileptic mode".
- **`noise_exc`**, **`noise_inh`**: Noise levels for excitatory and inhibitory neurons, repeated `copy_times` times.
- **`p`**: Connection probabilities between neuron types, repeated `copy_times` times, specifying the probability of connections from excitatory to excitatory (`pe2e`), from excitatory to inhibitory (`pe2i`), from inhibitory to excitatory (`pi2e`), and from inhibitory to inhibitory (`pi2i`).

### Stimulus Settings
- **`input_signal_file`**: List of filenames for the input signal files, repeated `copy_times` times. These files should contain the stimulus data used in the simulations.
- **`coord_of_stimulus`**, **`radius_of_stimulus`**: Coordinates and radius of the stimulus location, repeated `copy_times` times.

### Treatment Settings
- **`device_sensitivity`**: How frequently to check if the firing rate is above the threshold, repeated `copy_times` times.
- **`firing_rate_threshold`**: The threshold for activating treatment, repeated `copy_times` times.
- **`Eke_treatment`**, **`Eki_treatment`**: Excitability levels for excitatory and inhibitory neurons during treatment, repeated `copy_times` times.
- **`radius_of_electrode`**, **`distance_between_masks`**: Radius of the electrode and the distance between treatment masks, repeated `copy_times` times.

### Usage
This structure suggests that the `variables` dictionary is intended to be iterated over in a loop, where each iteration uses a subset of the dictionary corresponding to a single simulation run. The `copy_times` variable determines how many times the entire set of parameters is repeated, effectively creating multiple simulations with slight variations in parameters. This approach allows for systematic exploration of the parameter space to study the effects of different configurations on the neural network's behavior.


In [10]:
###############################
# Automated Runloop Variables #
###############################

copy_times = 5

# To activate epileptic mode, see "Eke_baseline" and "p" settings

variables = {
    
    # General Settings

    "run_id": ['Results 1', 'Results 2', 'Results 3', 'Results 4', 'Results 5'],
    "duration": [4000*ms]*copy_times,
    "bounds": [[600, 600, 600]]*copy_times, # [x_bound, y_bound, z_bound]
    
    # Network Settings
    
    "N": [[13500, 3375]]*copy_times, # [N_exc, N_inh]

    "Eke_baseline": [-84*mV]*copy_times,  # Set to =90 for epileptic mode
    "Eki_baseline": [-90*mV]*copy_times,

    "noise_exc": [[0.07, 0.075]*nA]*copy_times, #[0.1045, 0.104]
    "noise_inh": [[0.05, 0.08]*nA]*copy_times,

    # Normal ranges from 0.7-0.75, to activate sprouting increase the normal by 0.5
    # This will increase the average number of excitatory connections by 500
    "p": [[0.75, 0.35, 0.35, 0.0]]*copy_times, # pe2e, pe2i, pi2e, pi2i

    # Stimulus Settings

    "input_signal_file": ['sigmoid-1.0.txt']*copy_times, # files found in /stimuli/
    "coord_of_stimulus": [[300, 300, 300]]*copy_times,
    "radius_of_stimulus": [180]*copy_times,

    # Treatment Settings
    
    "device_sensitivity": [8*ms]*copy_times, # Device sensitivity - how frequently to check is firing rate is above the threshold
    "firing_rate_threshold": [5*Hz]*copy_times, # If firing rate goes above, the treatment activates

    "Eke_treatment": [-100*mV]*copy_times,
    "Eki_treatment": [-90*mV]*copy_times,

    "radius_of_electrode": [200]*copy_times,
    "distance_between_masks": [100]*copy_times,
    
}


In [11]:
run_model_loop(variables)


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100


Radius of electrode: 200
Max distance possible between masks: 173.20508075688772
Distance selected: 100



FileExistsError: [WinError 183] Eine Datei kann nicht erstellt werden, wenn sie bereits vorhanden ist: './results/Results 1'